# Extracción de datos: Accidentes de Tránsito (ATUS - INEGI)

## 1. Descripción general

Esta notebook forma parte del proyecto [**Relacion Baches Accidentes HMO**](https://github.com/Ingenieria-de-Caracteristicas-MCD/Relacion_Baches_Accidentes_HMO). 

En esta etapa se trabaja con el conjunto de datos **Accidentes de Tránsito Terrestre en Zonas Urbanas y Suburbanas (ATUS)**, publicado por el **INEGI**, disponible públicamente en:

🔗 [https://www.inegi.org.mx/programas/accidentes/](https://www.inegi.org.mx/programas/accidentes/)


El conjunto contiene información geoespacial y tabular sobre los accidentes registrados en zonas urbanas y suburbanas de México. Incluye atributos como:
- Fecha y hora del accidente  
- Tipo de vehículo involucrado  
- Causas probables  
- Coordenadas geográficas (X, Y)  
- Entidad y municipio

### 1.1 Objetivo de esta notebook

En esta notebook se documenta el proceso automatizado de:
1. **Descargar** los archivos comprimidos (ZIP) de ATUS para los años 2021–2023.  
2. **Extraer y filtrar** los datos correspondientes al municipio de **Hermosillo, Sonora (CVE_ENT = 26, CVE_MUN = 30)**.  
3. **Guardar** los archivos procesados en la estructura de carpetas del proyecto (`data/raw/` y `data/interim/`).

El flujo completo combina la ejecución de dos scripts principales:
- `src/download_atus.py`  
- `src/extract_atus.py`

## 2. Configuración del entorno

A continuación se importan las funciones y rutas necesarias para ejecutar el flujo completo de extracción de los datos ATUS. 

El archivo `src/config.py` define las rutas base del proyecto, mientas que los módulos `src/download_atus.py` y `src/extract_atus.py` contienen la lógica de descarga y extracción, respectivamente. 


In [1]:
import sys
from pathlib import Path

import pandas as pd
import geopandas as gpd

In [2]:
SRC_DIR = Path().resolve().parent / "src"

if str(SRC_DIR) not in sys.path: 
    sys.path.insert(0, str(SRC_DIR))

# for x in sys.path: print(x)

In [3]:
from config import ROOT_DIR, RAW_DIR, INTERIM_DIR
from download_atus import download_atus
from extract_atus import process_extraction_atus_hmo

# Verificación de rutas
print("Raw data directory:", RAW_DIR.relative_to(ROOT_DIR), RAW_DIR.exists(), RAW_DIR.is_dir())
print("Interim data directory:", INTERIM_DIR.relative_to(ROOT_DIR), INTERIM_DIR.exists(), INTERIM_DIR.is_dir())


ModuleNotFoundError: No module named 'config'

## 3. Descarga de datos ATUS

El script `download_atus.py` descarga en paralelo los archivos ZIP publicados por el INEGI para los años 2021, 2022 y 2023.  

Cada archivo se guarda en `data/raw/atus` y se valida su integridad antes de continuar. 

In [ ]:
# Descarga de archivos ZIP (2021–2023)
atus_zips = download_atus()

## 4. Extracción y filtrado de Hermosillo

El script `src/extract_atus.py` realiza las siguientes operaciones: 

1. Extrae todos los archivos ZIP descargados en `data/raw/atus`. 
2. Busca todos los archivos CSV generados durante la extracción. 
3. Filtra únicamente los regostros correspondientes al municipio de Hermosillo: 
    - `EDO` = 26 (Sonora)
    - `MPIO` = 30 (Hermosillo)
4. Guarda los resultados en `data/interim/atus`

* **Nota**: Si se ejecuta con el parámetro `clean=True`, se eliminarán ZIP originales una vez finalizado el proceso. 

In [ ]:
# Extracción y filtrado de Hermosillo
atus_filtered = process_extraction_atus_hmo(clean=False)

# 5. Verificación rápida del resultado

A continuación, se carga uno de los archivos CSV resultantes para verificar que la estructura y los campos sean correctos. 

En este punto, se espera que los datos correspondan únicamente a accidentes ocurridos en Hermosillo, Sonora. 

In [ ]:
df_hmo = pd.read_csv(atus_filtered[0])
df_hmo.head()

## 6. Ejecución desde la terminal

Alternativamente, si se prefiere ejecutar el flujo de descarga y extracción sin usar esta notebook, ambos scripts pueden correrse directamente desde la terminal.

### 6.1 Descargar los archivos ATUS (2021–2023)

```bash
python src/download_atus.py
````

Esto descargará los archivos ZIP oficiales del INEGI en la ruta: `data/raw/atus/`

### 6.2 Extraer y filtrar los datos de Hermosillo

```bash
python src/extract_atus.py
```

Este comando:

* Descomprime los archivos descargados en `data/raw/atus/`
* Filtra los registros de Hermosillo (`EDO=26`, `MPIO=30`)
* Guarda los resultados en `data/interim/atus/`
* Y si se ejecuta con la opción `clean=True` dentro del script, eliminará los ZIP originales.

### 6.3 Flujo completo

En caso de querer correr ambos pasos consecutivamente:

```bash
python src/download_atus.py && python src/extract_atus.py
```

## 7. Conclusión

En esta notebook se documentó el flujo completo de **extracción de los datos ATUS**:

- Se descargaron automáticamente los conjuntos de datos de INEGI (2021–2023).  
- Se extrajeron y filtraron los registros de Hermosillo.  
- Se generaron archivos intermedios limpios para su posterior análisis, limpieza e integración con los datos de **baches reportados (Bachómetro)** y **clima (Open Meteo)**.

Los datos procesados servirán como base para el análisis espacial y temporal de accidentes y su relación con condiciones viales y meteorológicas.
